In [1]:
from transformers import pipeline
from datasets import load_dataset
import requests
from PIL import Image
import os
import torch
import time
from tqdm import tqdm
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration

/home/govind/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-01 21:15:51.537809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 21:15:52.059526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-01 21:15:52.059565: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dle

In [2]:
DATA_DIR = "/scratch/govind/modality_translation/images_thermal_train/data"
files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if os.path.isfile(os.path.join(DATA_DIR, f))]

In [3]:
model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.13it/s]


In [4]:
d = {}

t0 = time.time()

files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if os.path.isfile(os.path.join(DATA_DIR, f))]
loader = torch.utils.data.DataLoader(files, batch_size=8)
for i, batch in enumerate(loader):
    prompt = "USER: <image>\nGive a very concise caption for the image.\nASSISTANT:"
    inputs = processor([prompt for _  in batch], images=[Image.open(file) for file in batch], padding=True, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=200)
    completed_prompts = processor.batch_decode(output, skip_special_tokens=True)

    captions = [completed_prompt.split('ASSISTANT: ')[1].lower() for completed_prompt in completed_prompts]
    
    print("iter/sec:", len(batch)*(i+1) / (time.time() - t0))

    for f, c in zip(batch, captions):
        print(f)
        print(c)
        d[f] = c

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


iter/sec: 1.1313613135247962
/scratch/govind/modality_translation/images_thermal_train/data/video-AEZAaF8epmvQv5Nfj-frame-009629-g3ckpghgD5CRJqHrM.jpg
"a group of people walking down a street at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-ELruYFQjkqpZnir29-frame-003174-eecyBKvvyQ75MxFrX.jpg
"nighttime street scene with cars and traffic lights." 
/scratch/govind/modality_translation/images_thermal_train/data/video-TTDq3x6HSvFx3YRDH-frame-001049-NwrB4sbE2APoLjS8Z.jpg
"a long, empty street." 
/scratch/govind/modality_translation/images_thermal_train/data/video-TTDq3x6HSvFx3YRDH-frame-001169-f4GPmW6AE3rbnHhCF.jpg
"a black and white photo of a city street." 
/scratch/govind/modality_translation/images_thermal_train/data/video-ZicHYAPRYGdu82nAa-frame-003471-g2gvndmv8n7EoycbH.jpg
"a person riding a bicycle at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-ZicHYAPRYGdu82nAa-frame-001912-x9YviHBfWd9xyHhzD.jpg
"nighttime city stree

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


iter/sec: 1.1494921160623348
/scratch/govind/modality_translation/images_thermal_train/data/video-KTtJnXH7CcPgLeF2G-frame-005006-n96Hm3sYm7oXJEJa4.jpg
"a person walking in a parking lot at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-vvYEi6EdCrYXeHwpD-frame-015300-Q7Wtamhk3mpErurDE.jpg
"nighttime traffic on a highway." 
/scratch/govind/modality_translation/images_thermal_train/data/video-7yaY3LisuPdNyfZH9-frame-004335-PXAKWPrHfvpBn92mx.jpg
"a quiet, snowy street at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-aiXLPEvp86raemrsp-frame-003825-aTd67CNNGNqWMmoqi.jpg
"a city street at night with a double-decker bus and cars." 
/scratch/govind/modality_translation/images_thermal_train/data/video-BjSfmxLQqCGjg8tya-frame-004469-YP4WajtXu5keDi3uh.jpg
"nighttime city street with traffic and pedestrians." 
/scratch/govind/modality_translation/images_thermal_train/data/video-yBdrow6zFe5E8uEwn-frame-006020-cyLajDhMFnWdozS4K.jpg
"a cit

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


iter/sec: 1.162199734225654
/scratch/govind/modality_translation/images_thermal_train/data/video-DJYDQTZH7GLCqw3Tx-frame-008780-ELH7X6s5BzggWaDss.jpg
"a black and white photo of a person riding a bicycle on a city street." 
/scratch/govind/modality_translation/images_thermal_train/data/video-zNFzcc9wW8XB4QwTa-frame-003567-BL2rumRamXgxsjwZu.jpg
"a quiet street at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-KTtJnXH7CcPgLeF2G-frame-002988-c6xG7dMX3gbS6g44b.jpg
"a black and white photo of a street at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-evnmmkhaqckdxWa9P-frame-002266-pNaXToLcA3s6DFyz5.jpg
"a train on a track at night." 
/scratch/govind/modality_translation/images_thermal_train/data/video-4E8nAe7bkHkSZfpjq-frame-008700-a4SMFQgdw9YyqcCvb.jpg
"a blurry nighttime street scene with palm trees and a car." 
/scratch/govind/modality_translation/images_thermal_train/data/video-muKWZNM2FxQoStMQh-frame-000079-v4JW5QvY8uL7M2xES

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

: 